In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

gender_submission.csv
test.csv
train.csv



In [17]:
df_test = pd.read_csv("../input/test.csv")
df_train = pd.read_csv("../input/train.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
#Converts Embarked location to numbers to enable correlation calcluation
df_train["Embarked"] = df_train["Embarked"].replace("C",1)
df_train["Embarked"] = df_train["Embarked"].replace("Q",2)
df_train["Embarked"] = df_train["Embarked"].replace("S",3)
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,3.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,3.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,3.0


In [39]:
df_train["Sex"] = df_train["Sex"].replace("male",1)
df_train["Sex"] = df_train["Sex"].replace("female",0)
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3.0


In [45]:
df_train_num = df_train.iloc[:,0:3]
df_train_num["Sex"] = df_train["Sex"] 
df_train_num["SibSp"] = df_train["SibSp"] 
df_train_num["Parch"] = df_train["Parch"] 
df_train_num["Fare"] = df_train["Fare"] 
df_train_num["Embarked"] = df_train["Embarked"] 
df_train_num.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,1,0,3,1,1,0,7.2500,3.0
1,2,1,1,0,1,0,71.2833,1.0
2,3,1,3,0,0,0,7.9250,3.0
3,4,1,1,0,1,0,53.1000,3.0
4,5,0,3,1,0,0,8.0500,3.0


In [49]:
corr = df_train_num.corr()

In [50]:
print(corr)

             PassengerId  Survived    ...         Fare  Embarked
PassengerId     1.000000 -0.005007    ...     0.012658  0.013166
Survived       -0.005007  1.000000    ...     0.257307 -0.169718
Pclass         -0.035144 -0.338481    ...    -0.549500  0.164681
Sex             0.042939 -0.543351    ...    -0.182333  0.110320
SibSp          -0.057527 -0.035322    ...     0.159651  0.068900
Parch          -0.001652  0.081629    ...     0.216225  0.040449
Fare            0.012658  0.257307    ...     1.000000 -0.226311
Embarked        0.013166 -0.169718    ...    -0.226311  1.000000

[8 rows x 8 columns]
